In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz
!tar xf spark-3.1.1-bin-hadoop3.2.tgz
!pip install -q findspark

In [2]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"

In [3]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
spark.conf.set("spark.sql.repl.eagerEval.enabled", True) # Property used to format output tables better
spark

In [4]:
from google.colab import files
uploaded=files.upload()

Saving earthquake.csv to earthquake.csv


In [56]:
df=spark.read.format("csv").load("earthquake.csv",header=True)
df.show()

+-------------+-------------------+--------+---------+-----+---------+------------------+---------------+--------------------+----------+
|earthquake_id|        occurred_on|latitude|longitude|depth|magnitude|calculation_method|     network_id|               place|     cause|
+-------------+-------------------+--------+---------+-----+---------+------------------+---------------+--------------------+----------+
|            1| 1969-01-01 9:07:06|  51.096| -179.392|   45|      5.6|                mw|   iscgem812771|Andreanof Islands...|earthquake|
|            2|1969-01-02 17:50:48| -56.096|  -27.842| 80.1|        6|                mw|iscgemsup812819|South Sandwich Is...|earthquake|
|            3| 1969-01-03 3:16:40|   37.14|   57.899|   10|      5.5|                mw|   iscgem812826|Turkmenistan-Iran...|earthquake|
|            4|1969-01-03 13:28:12|  51.132| -179.306|   15|      5.9|                mw|   iscgem812841|Andreanof Islands...|earthquake|
|            5|1969-01-04 22:36:48

In [58]:
#greatest()
df.printSchema()
from pyspark.sql.types import IntegerType
#df = df.withColumn("SAMPLE_GREATER", df["SAMPLE_GREATER"].cast(IntegerType()))
df = df.withColumn("latitude", df["latitude"].cast(IntegerType()))
df = df.withColumn("longitude", df["longitude"].cast(IntegerType()))
from pyspark.sql.functions import greatest,col
df1=df.withColumn("SAMPLE_GREATER",greatest(col("latitude"),col("longitude")))            #doubt
df1.show()


root
 |-- earthquake_id: string (nullable = true)
 |-- occurred_on: string (nullable = true)
 |-- latitude: integer (nullable = true)
 |-- longitude: integer (nullable = true)
 |-- depth: string (nullable = true)
 |-- magnitude: string (nullable = true)
 |-- calculation_method: string (nullable = true)
 |-- network_id: string (nullable = true)
 |-- place: string (nullable = true)
 |-- cause: string (nullable = true)

+-------------+-------------------+--------+---------+-----+---------+------------------+---------------+--------------------+----------+--------------+
|earthquake_id|        occurred_on|latitude|longitude|depth|magnitude|calculation_method|     network_id|               place|     cause|SAMPLE_GREATER|
+-------------+-------------------+--------+---------+-----+---------+------------------+---------------+--------------------+----------+--------------+
|            1| 1969-01-01 9:07:06|      51|     -179|   45|      5.6|                mw|   iscgem812771|Andreanof Islan

In [59]:
#least()
#cast()
from pyspark.sql.functions import least,col
df2=df.withColumn("SAMPLE_LEAST",least(col("latitude"),col("longitude")))
df2.show()

+-------------+-------------------+--------+---------+-----+---------+------------------+---------------+--------------------+----------+------------+
|earthquake_id|        occurred_on|latitude|longitude|depth|magnitude|calculation_method|     network_id|               place|     cause|SAMPLE_LEAST|
+-------------+-------------------+--------+---------+-----+---------+------------------+---------------+--------------------+----------+------------+
|            1| 1969-01-01 9:07:06|      51|     -179|   45|      5.6|                mw|   iscgem812771|Andreanof Islands...|earthquake|        -179|
|            2|1969-01-02 17:50:48|     -56|      -27| 80.1|        6|                mw|iscgemsup812819|South Sandwich Is...|earthquake|         -56|
|            3| 1969-01-03 3:16:40|      37|       57|   10|      5.5|                mw|   iscgem812826|Turkmenistan-Iran...|earthquake|          37|
|            4|1969-01-03 13:28:12|      51|     -179|   15|      5.9|                mw|   is

In [30]:
#split()

from pyspark.sql.functions import split
yes=df2.withColumn('date',split(col('occurred_on'),' ').getItem(0)).withColumn('time',split(col('occurred_on'), ' ').getItem(1))
yes.show()

+-------------+-------------------+--------+---------+-----+---------+------------------+---------------+--------------------+----------+------------+----------+--------+
|earthquake_id|        occurred_on|latitude|longitude|depth|magnitude|calculation_method|     network_id|               place|     cause|SAMPLE_LEAST|      date|    time|
+-------------+-------------------+--------+---------+-----+---------+------------------+---------------+--------------------+----------+------------+----------+--------+
|            1| 1969-01-01 9:07:06|  51.096| -179.392|   45|      5.6|                mw|   iscgem812771|Andreanof Islands...|earthquake|    -179.392|1969-01-01| 9:07:06|
|            2|1969-01-02 17:50:48| -56.096|  -27.842| 80.1|        6|                mw|iscgemsup812819|South Sandwich Is...|earthquake|     -27.842|1969-01-02|17:50:48|
|            3| 1969-01-03 3:16:40|   37.14|   57.899|   10|      5.5|                mw|   iscgem812826|Turkmenistan-Iran...|earthquake|        

In [31]:
df = spark.createDataFrame([["PRITAM", 24], ["NIKITA", 25],["RISHAV",22]], ["NAME", "AGE"])
df.show()

+------+---+
|  NAME|AGE|
+------+---+
|PRITAM| 24|
|NIKITA| 25|
|RISHAV| 22|
+------+---+



In [32]:
#lower()

import pyspark.sql.functions as F
df.select(F.lower(df.NAME)).show()

+-----------+
|lower(NAME)|
+-----------+
|     pritam|
|     nikita|
|     rishav|
+-----------+



In [33]:
#replacing columnn with lowercased column in pyspark

import pyspark.sql.functions as F
df.withColumn("name", F.lower(df.NAME)).show()

+------+---+
|  name|AGE|
+------+---+
|pritam| 24|
|nikita| 25|
|rishav| 22|
+------+---+



In [60]:
df = spark.createDataFrame([["Pritam", 24], ["Nikita", 25],["Sunita",23],["Pritam", 24], ["Nikita", 11],["Sunita",67]],["Name", "Age"])
df.show()

+------+---+
|  Name|Age|
+------+---+
|Pritam| 24|
|Nikita| 25|
|Sunita| 23|
|Pritam| 24|
|Nikita| 11|
|Sunita| 67|
+------+---+



In [35]:
#max() using select()

from pyspark.sql.functions import max
df.select(max(df.Age)).show()

+--------+
|max(Age)|
+--------+
|      67|
+--------+



In [36]:
#max() using groupBy

df.groupBy("Name").max() \
     .show()

+------+--------+
|  Name|max(Age)|
+------+--------+
|Sunita|      67|
|Pritam|      24|
|Nikita|      25|
+------+--------+



In [37]:
#mean()

df2.agg({'depth': 'mean'}).show()

+----------------+
|      avg(depth)|
+----------------+
|71.1652558934211|
+----------------+



In [38]:
#round()
#groupBy()

from pyspark.sql.functions import round, col
df2.select("*", round(col('longitude'),2)).show()                  #2 indicates upto 2 decimal places

+-------------+-------------------+--------+---------+-----+---------+------------------+---------------+--------------------+----------+------------+-------------------+
|earthquake_id|        occurred_on|latitude|longitude|depth|magnitude|calculation_method|     network_id|               place|     cause|SAMPLE_LEAST|round(longitude, 2)|
+-------------+-------------------+--------+---------+-----+---------+------------------+---------------+--------------------+----------+------------+-------------------+
|            1| 1969-01-01 9:07:06|  51.096| -179.392|   45|      5.6|                mw|   iscgem812771|Andreanof Islands...|earthquake|    -179.392|            -179.39|
|            2|1969-01-02 17:50:48| -56.096|  -27.842| 80.1|        6|                mw|iscgemsup812819|South Sandwich Is...|earthquake|     -27.842|             -27.84|
|            3| 1969-01-03 3:16:40|   37.14|   57.899|   10|      5.5|                mw|   iscgem812826|Turkmenistan-Iran...|earthquake|        

In [39]:
#collect_set()

from pyspark.sql.functions import collect_set
df2.agg(collect_set('magnitude')).collect()

[Row(collect_set(magnitude)=['5.58', '5.72', '6.2', '9.1', '7', '5.82', '5.53', '5.94', '7.4', '8.3', '8.2', '6.6', '5.55', '5.75', '6.02', '7.1', '6.5', '5.66', '7.9', '5.67', '5.84', '6.7', '8.1', '5.97', '7.6', '5.64', '5.88', '6.9', '8', '5.9', '8.4', '5.81', '8.8', '7.2', '6.57', '5.7', '5.73', '6.3', '5.5', '5.77', '5.54', '5.52', '5.6', '6.1', '5.8', '5.69', '7.8', '7.7', '8.6', '6.8', '6.4', '6.45', '7.5', '5.93', '7.3', '5.89', '5.51', '6'])]

In [40]:
#collect_list()

from pyspark.sql.functions import collect_list
df2.agg(collect_list('depth')).collect()

[Row(collect_list(depth)=['45', '80.1', '10', '15', '105', '60', '10', '45', '25', '50', '25', '130', '244.7', '112.9', '10', '25', '30.1', '593.5', '35', '31.3', '15', '15', '60', '43.3', '40', '260', '35', '45', '45', '30', '35', '45', '35', '660', '662', '13.3', '430.5', '210.1', '40', '75', '55', '35', '15', '35', '65', '10', '10', '15', '34.5', '35', '205.6', '170', '20', '20', '198.7', '179.8', '20', '42.3', '38.8', '12.9', '168.5', '10', '20', '20', '17.1', '10', '15', '15', '15', '15', '50', '10', '10', '10', '15', '10', '398.1', '15', '15', '15.9', '15', '15', '15', '15', '15', '110.4', '565', '20', '167.5', '10', '588.3', '25', '100', '35', '10', '30', '70.7', '38.6', '10', '34.5', '17.9', '27.4', '35', '35', '35', '45', '283.5', '68.6', '203.4', '10', '75', '50', '631.7', '28.3', '30', '34.8', '30', '30', '15', '60', '15', '30', '158.6', '20', '20', '15', '15', '22.4', '55', '55', '202.6', '15', '32.7', '40', '17.8', '557.1', '15', '56.7', '20', '100', '15', '35', '10', '10'

In [41]:
#concat() 

from pyspark.sql.functions import concat,col
data = [('    Pritam    ','','Das','16/10/98','M',3000),
  ('Nikita      ','','Panda','10/12/97','F',4000),
  ('    piu','Kumar','','03/02/98','M',4000)
]

columns = ["first_name","middle_name","last_name","dob","gender","salary"]

In [42]:
df = spark.createDataFrame(data=data, schema = columns)
df2=df.select(concat(df.first_name,df.middle_name,df.last_name)
              .alias("FullName"),"dob","gender","salary")
df2.show(truncate=False)

+-----------------+--------+------+------+
|FullName         |dob     |gender|salary|
+-----------------+--------+------+------+
|    Pritam    Das|16/10/98|M     |3000  |
|Nikita      Panda|10/12/97|F     |4000  |
|    piuKumar     |03/02/98|M     |4000  |
+-----------------+--------+------+------+



In [43]:
#concat_ws()

from pyspark.sql.functions import concat_ws,col
df3=df.select(concat_ws('_',df.first_name,df.middle_name,df.last_name)
              .alias("FullName"),"dob","gender","salary")
df3.show(truncate=False)

+-------------------+--------+------+------+
|FullName           |dob     |gender|salary|
+-------------------+--------+------+------+
|    Pritam    __Das|16/10/98|M     |3000  |
|Nikita      __Panda|10/12/97|F     |4000  |
|    piu_Kumar_     |03/02/98|M     |4000  |
+-------------------+--------+------+------+



In [49]:
#date_format()
from pyspark.sql.functions import *
yes.select(col("date"), 
    date_format(col("date"), "MM-dd-yyyy").alias("date_format") 
  ).show()

+----------+-----------+
|      date|date_format|
+----------+-----------+
|1969-01-01| 01-01-1969|
|1969-01-02| 01-02-1969|
|1969-01-03| 01-03-1969|
|1969-01-03| 01-03-1969|
|1969-01-04| 01-04-1969|
|1969-01-05| 01-05-1969|
|1969-01-06| 01-06-1969|
|1969-01-08| 01-08-1969|
|1969-01-10| 01-10-1969|
|1969-01-13| 01-13-1969|
|1969-01-14| 01-14-1969|
|1969-01-18| 01-18-1969|
|1969-01-19| 01-19-1969|
|1969-01-19| 01-19-1969|
|1969-01-20| 01-20-1969|
|1969-01-20| 01-20-1969|
|1969-01-22| 01-22-1969|
|1969-01-24| 01-24-1969|
|1969-01-25| 01-25-1969|
|1969-01-26| 01-26-1969|
+----------+-----------+
only showing top 20 rows



In [50]:
#dayofweek(), dayofmonth(), dayofyear()

yes.select(col("date"),  
     dayofweek(col("date")).alias("dayofweek"), 
     dayofmonth(col("date")).alias("dayofmonth"), 
     dayofyear(col("date")).alias("dayofyear"), 
  ).show()

+----------+---------+----------+---------+
|      date|dayofweek|dayofmonth|dayofyear|
+----------+---------+----------+---------+
|1969-01-01|        4|         1|        1|
|1969-01-02|        5|         2|        2|
|1969-01-03|        6|         3|        3|
|1969-01-03|        6|         3|        3|
|1969-01-04|        7|         4|        4|
|1969-01-05|        1|         5|        5|
|1969-01-06|        2|         6|        6|
|1969-01-08|        4|         8|        8|
|1969-01-10|        6|        10|       10|
|1969-01-13|        2|        13|       13|
|1969-01-14|        3|        14|       14|
|1969-01-18|        7|        18|       18|
|1969-01-19|        1|        19|       19|
|1969-01-19|        1|        19|       19|
|1969-01-20|        2|        20|       20|
|1969-01-20|        2|        20|       20|
|1969-01-22|        4|        22|       22|
|1969-01-24|        6|        24|       24|
|1969-01-25|        7|        25|       25|
|1969-01-26|        1|        26

In [51]:
#trim()
#remove white spaces

df2.withColumn("FullName",trim(col("FullName"))).show()

+-----------------+--------+------+------+
|         FullName|     dob|gender|salary|
+-----------------+--------+------+------+
|    Pritam    Das|16/10/98|     M|  3000|
|Nikita      Panda|10/12/97|     F|  4000|
|         piuKumar|03/02/98|     M|  4000|
+-----------------+--------+------+------+



In [52]:
df4=df2.withColumn('new_gender', when(col('gender')=='M','Male').otherwise('Female'))
df4.show()

+-----------------+--------+------+------+----------+
|         FullName|     dob|gender|salary|new_gender|
+-----------------+--------+------+------+----------+
|    Pritam    Das|16/10/98|     M|  3000|      Male|
|Nikita      Panda|10/12/97|     F|  4000|    Female|
|         piuKumar|03/02/98|     M|  4000|      Male|
+-----------------+--------+------+------+----------+



In [61]:
df4.drop('gender')

FullName,dob,salary,new_gender
Pritam Das,16/10/98,3000,Male
Nikita Panda,10/12/97,4000,Female
piuKumar,03/02/98,4000,Male


In [62]:
#translate()

data = spark.createDataFrame([["Pritam", 24], ["Nikita", 20], ["Rahul", 40]], ["name", "age"])
data.show()

+------+---+
|  name|age|
+------+---+
|Pritam| 24|
|Nikita| 20|
| Rahul| 40|
+------+---+



In [65]:
import pyspark.sql.functions as F
data.select(F.translate("name", "PriR", "ooot")).show()               #in name column P-->o
                                                                      #r-->o
                                                                      #i-->o
                                                                      #R-->T

+---------------------------+
|translate(name, PriR, ooot)|
+---------------------------+
|                     oootam|
|                     Nokota|
|                      tahul|
+---------------------------+

